# QnA

In [10]:
import os
import openai
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-09-15-preview"

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [22]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [ ]:
file = 'QAData.csv'
loader = CSVLoader(file_path=file)

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

response = index.query(query)

display(Markdown(response))

# FIXME
# Above is based on OpenAI API, but it is not working for me yet. 
# Because I use Azure API


## Step by Step

In [30]:
from langchain.document_loaders import CSVLoader
file = 'QAData.csv'
loader = CSVLoader(file_path=file)

In [31]:
docs = loader.load()

In [32]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'QAData.csv', 'row': 0})

In [33]:
from langchain.embeddings import OpenAIEmbeddings
embedding = OpenAIEmbeddings(deployment="embedding")

In [34]:
embed = embedding.embed_query("Hi my name is John")

In [35]:
len(embed)

1536

In [36]:
embed[:10]

[-0.011283237498878494,
 0.013887061752001818,
 -0.03203943375123669,
 -0.03055153604066851,
 -0.02988198216404509,
 0.026311023188333102,
 -0.006227479322807317,
 0.009745741273224721,
 -0.009206377910765534,
 0.00021214192406109888]

In [38]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embedding
)

In [39]:
query = "Please suggest a shirt with sunblocking"

In [43]:
res = db.similarity_search(query)
len(docs)

1000

In [44]:
res[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'QAData.csv', 'row': 255})

In [ ]:
retriever = db.as_retriever()